<a href="https://colab.research.google.com/github/FlowSight/MlAlgoFromScratch/blob/master/regression_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We need a regression-useable dataset

In [ ]:
# gdrive_mount_path='/content/gdrive'
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#%cd /content/gdrive/My\ Drive/Colab\ Notebooks/practices
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt

--2020-02-23 17:36:38--  https://archive.ics.uci.edu/ml/machine-learning-databases/00246/3D_spatial_network.txt
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20673913 (20M) [application/x-httpd-php]
Saving to: ‘3D_spatial_network.txt’

3D_spatial_network. 100%[===================>]  19.72M  11.3MB/s    in 1.7s    

2020-02-23 17:36:40 (11.3 MB/s) - ‘3D_spatial_network.txt’ saved [20673913/20673913]



In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
dataset = pd.read_csv("3D_spatial_network.txt", sep=",", header=None)
from sklearn.utils import shuffle
dataset = shuffle(dataset)
X=dataset.iloc[:,1:3].to_numpy()
Y=dataset.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
X_train=np.array(X_train).reshape((X_train.shape[0],X_train.shape[1]))
y_train=np.array(y_train).reshape((y_train.shape[0],1))

In [ ]:
 # This would do linear and ridge regression conditionally
 class LinearRegressionCustom:

  def __init__(self,x,y, epochs=10000,learning_rate=0.0001,ridge=False,ridgeCoef=1.0,printEpochInterval=100):  
        self.x = x
        self.y=y
        self.n_train=self.x.shape[0]
        self.m=self.x.shape[1]
        self.epochs=epochs
        self.printEpochInterval=printEpochInterval
        assert(printEpochInterval>=5)
        self.learning_rate=learning_rate
        self.ridge=ridge
        self.ridgeCoef=ridgeCoef
        if ridge==False:
          self.ridgeCoef=0.0
        self.bias=0.0
        self.weights=np.full((self.m, 1), 1.0)
        self.best_weights=np.full((self.m, 1), 1.0)
        self.best_error=2.0**30
        assert(y.shape[1]==1)
        assert(x.shape[0]==y.shape[0])
        self.errors = np.zeros(self.epochs)
        self.LinearRegression()

  def LSEGradDes(self):
   # initialize stuff
   error=0.0
   gradients=np.full((self.m, 1), 0.0)
   bias_grad=0
   
   y_pred = np.add(self.bias,np.dot(self.x,self.weights))
   error=np.square(np.subtract(self.y, y_pred)).mean(axis=0) # SE error
   gradients = -2/self.n_train*(self.x.T.dot(np.subtract(self.y,y_pred))+self.ridgeCoef*self.best_weights)  # Derivative wrt weights
   bias_grad = -2/self.n_train*np.sum(np.subtract(self.y, y_pred),axis=0)  # Derivative wrt bias
   return gradients,bias_grad,error
  
  def LinearRegression(self):
    # start action in epochs
    for epoch in range(self.epochs):
      gradients,bias_grad,epoch_loss = [],0.0,0.0
      gradients,bias_grad,epoch_loss=self.LSEGradDes()
      
      # update weights
      best_error=2.0**30
      self.bias = self.bias - self.learning_rate * bias_grad#bias update
      self.weights = np.subtract(self.weights,self.learning_rate * gradients)
      self.errors[epoch]=epoch_loss
      if epoch_loss < np.amin(self.errors):
        self.best_weights=self.best_weights
        self.best_error=epoch_loss
      if epoch%self.printEpochInterval==0 :
        print("epoch: "+str(epoch+1))
        print("weights: "+str(self.weights))
        print("bias: "+str(self.bias))
        print("loss: "+str(epoch_loss))

  # this will return result and mse error as a tuple
  def predict(self,X_test):
    y_pred = np.add(self.bias,np.dot(self.x,self.weights))
    #error=np.square(np.subtract(y_test, y_pred)).mean(axis=0)
    return y_pred

In [ ]:
LinearRegressionCustom(X_train,y_train,learning_rate=0.0001,epochs=200,printEpochInterval=20)

In [ ]:
from collections import OrderedDict
from scipy import linalg
import numpy as np
from itertools import combinations_with_replacement


class PolynomialRegressionCustom(object):

    def __init__(self, x, y): 
        self.x = x
        self.y = y    

    def normalize(self,v):
     return v / np.sqrt(v.dot(v))

    def gramSchmidt(self,X):
      n = X.shape[1]
      X[:, 0] = self.normalize(X[:, 0])
      for i in range(1, n):
        print(i)
        Xi = X[:, i]
        for j in range(0, i):
            Xj = X[:, j]
            Xi = Xi - (Xi.dot(Xj))* Xj
        X[:, i] = self.normalize(Xi)
      return X

    def index_combinations(self):
        combinations = [combinations_with_replacement(range(self.n_features), i) for i in range(0, self.order + 1)]
        flat_combs = [item for sublist in combinations for item in sublist]
        return flat_combs

    def polynomial_transform(self,X, degree):
        n_samples, self.n_features = np.shape(X)
        combinations = self.index_combinations()
        n_output_features = len(combinations)
        X_new = np.empty((n_samples, n_output_features))
        
        for i, index_combs in enumerate(combinations):  
            X_new[:, i] = np.prod(X[:, index_combs], axis=1)

        return X_new       
        
    def fit(self, method = 'ne', order = 1,epochs = 60, learningRate = 0.01,printEpochInterval=100,ridge=False):
        theta=0
        self.order=order
        self.method=method
        tranformed_x =self.gramSchmidt(self.polynomial_transform(self.x,self.order))
        if method == 'ne': 
            theta = np.matmul(np.matmul(linalg.pinv(np.matmul(np.transpose(tranformed_x),tranformed_x)), np.transpose(tranformed_x)), self.y)

        elif method == 'gd': 
            lr = LinearRegressionCustom(tranformed_x,self.y,learning_rate=learningRate,epochs=epochs,printEpochInterval=printEpochInterval,ridge=ridge)
            self.model=lr 
        self.theta = theta

    def predict(self,X_test):
       tranformed_x =self.gramSchmidt(self.polynomial_transform(X_test,self.order))
       if self.method == 'ne':
         return self.theta*X_test

       elif self.method == 'gd':
         return self.model.predict(X_test)

In [ ]:
from __future__ import division, print_function
import numpy as np
import math

def normal_density(x,mean,var,eps=1e-4):
   return (1.0 / math.sqrt(2.0 * math.pi * var + eps))* math.exp(-(math.pow(x - mean, 2)/ (2 * var + eps)))

class NaiveBayes():
    def fit(self, X, y):
        self.X, self.y = X, y
        self.classes = np.unique(y)
        assert(self.classes <= 10)
        self.parameters = []
        for i, c in enumerate(self.classes):
            X_where_c = X[np.where(y == c)]
            self.parameters.append([])
            colidx=0
            for col in X_where_c.T:
                parameters = {"mean"+str(colidx): col.mean(), "var"+str(colidx): col.var()}
                self.parameters[i].append(parameters)
                colidx+=1

    def likelihood(self,mean, var, x,kernel="gaussian"):
        assert(kernel=="gaussian")
        if kernel=="gaussian":
          return normal_density(x,mean,var)

    def classify(self, sample):
        posteriors = []
        for i, c in enumerate(self.classes):
            prior = np.mean(self.y == c)
            # p(y|x) = p(y)*p(x1|y)*p(x2|y)*p(x3|y)...
            posterior = prior
            for feature_value, params in zip(sample, self.parameters[i]):
                likelihood = self.likelihood(params["mean"], params["var"], feature_value)
                posterior *= likelihood
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        y_pred = [self.classify(sample) for sample in X]
        return y_pred

In [ ]:
def linear_kernel(x1, x2):
      return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
      return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
      return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

def rbf_kernel(x1, x2,gamma=0.1):
     distance = np.linalg.norm(x1 - x2) ** 2
     return np.exp(-gamma * distance)

class SVM(object):
    
    def __init__(self, kernel=linear_kernel, C=None,p=None,gamma=None,sigma=None,tol=1e-5):
        self.kernel = kernel
        self.C = C
        self.tol=tol
        self.p=p
        self.gamma=gamma
        self.sigma=sigma
        if self.C is not None: self.C = float(self.C)
        if self.p is not None: self.p = float(self.p)
        if self.sigma is not None: self.sigma = float(self.sigma)
        if self.gamma is not None: self.gamma = float(self.gamma)
        if self.kernel is gaussian_kernel :
          assert(self.sigma is not None)
        if self.kernel is polynomial_kernel :
          assert(self.p is not None)
        if self.kernel is rbf_kernel :
          assert(self.gamma is not None)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        #1) We will first get lagrangians
        # 2) then we will have W, but we will calculate it runtime w*x=sum(lagrangian[i]*y[i]*<x[i],x[j]>)
        # 3) we can get bias b = sum(yi-w*xi)/total_elements
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
              if self.kernel is polynomial_kernel:
                K[i,j] = self.kernel(X[i], X[j],p=self.p)
              elif self.kernel is gaussian_kernel:
                K[i,j] = self.kernel(X[i], X[j],sigma=self.sigma)
              elif self.kernel is rbf_kernel:
                K[i,j] = self.kernel(X[i], X[j],gamma=self.gamma)
              else:
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1) # creating a nx1 -1 matrix
        A = cvxopt.matrix(y, (1,n_samples)) # this is basically reshaping y into (1xn)
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1) # diagonal matrix of -1 sized nxn
            tmp2 = np.identity(n_samples) # daigonal matrix of 1 sized nxn
            G = cvxopt.matrix(np.vstack((tmp1, tmp2))) # stack tmp1 and tmp2 vertically
            h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))) # horizontally stack tmp1 and tmp2

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        a = np.ravel(solution['x'])
        assert(len(a)==X.shape[0])

        # Support vectors have non zero lagrange multipliers
        sv = a > self.tol
        ind = np.arange(len(a))[sv]
        print(ind)
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print (str(len(self.a))+" support vectors out of "+str(n_samples) +" points")

        # b=sum(yi-w*xi)/n
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            y_predict = np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            y_predict = y_predict + self.b

        return np.sign(y_predict)